### Question 13.2

In this problem, you will simulate a simplified airport security system at a busy airport. Passengers arrive according to a Poisson distribution with $\lambda_1 = 5$ per minute (i.e., mean interarrival rate $\mu_1 = 0.2$ minutes) to the ID/boarding-pass check queue, where there are several servers who each have exponential service time with mean rate $\mu_2 = 0.75$ minutes. [Hint: model them as one block that has more than one resource.] After that, the passengers are assigned to the shortest of the several personal-check queues, where they go through the personal scanner (time is uniformly distributed between $0.5$ minutes and $1$ minute). 

Use the Arena software (PC users) or Python with SimPy (PC or Mac users) to build a simulation of the system, and then vary the number of ID/boarding-pass checkers and personal-check queues to determine how many are needed to keep average wait times below 15 minutes. Note: If you’re using SimPy, or if you have access to a non-student version of Arena, you can use $\lambda_1 = 50$ to simulate a busier airport.

### Results summary

For the simulation where the mean passenger arrival rate is 5, we find that the optimal configuration yielding mean wait time just below 15 minutes was with 5 id scanners and 9 personal checkers (time of 14.639845305).

For the simulation where the mean passenger arrival rate is 50, we find that the optimal configuration yielding mean wait time just below 15 minutes was with 35 id scanners and 31 personal checkers (time of 14.9797373981).

However, we would need to include costs in our final analysis as well. If personal checkers are twice as expensive as id scanners, we would choose a solution that has fewer personal checkers (for example num_id= 36  and num_pc= 20 yields a time of 14.7139433891).


### References

Example with analogous arrival processes and queues:

https://pythonhosted.org/SimPy/Tutorials/TheBank.html

TA carwash example: http://simpy.readthedocs.io/en/latest/examples/carwash.html

In [14]:
"""
Double-Queue airport security system
"""
import random
import simpy
import numpy as np

RANDOM_SEED = 42
TOTAL_WAIT=[]     #Total wait time to use both IDboarding and PersonalCheck stations
REPLICATIONS=5   #Number of replications for which to repeat the experiment
MU_1 = 0.2        #Mean interarrival rate for passenger generation
MU_2 = 0.75       #Mean service time (IDboarding check time) 


#IDboarding parameters. One queue for all passengers:
NUM_ID = 5         # Number of stations in the IDboarding check system
SIM_TIME = 100     # Simulation time in minutes (100 minutes was suggested in TA video)

# When the passengers are done at the IDboarding station, they enter the 
# PersonalCheck station where there is a queue for each PersonalCheck machine

#PersonalCheck parameters
NUM_PC = 5  #Number of PersonalCheck stations
MIN_PC = 0.5 #Lower bound on PersonalCheck time
MAX_PC = 1.0 #Upper bound on PersonalCheck time


#Resource queue counter
def NoInSystem(R):                                                  
    """ Total number of customers in the resource R"""
    return (len(R.queue)+R.count)                            


#Define IDboarding class
class IDboarding(object):
    """An IDboarding has a limited number of stations (``NUM_ID``) to
    check passenger ID's in parallel.

    Passengers have to request one of the machines. When they get one, 
    they can start the checking processes and wait for it to finish 
    (which takes an average of ``mu_2`` minutes).

    """
    def __init__(self, env, num_id, mu_2):
        self.env = env
        self.machine = simpy.Resource(env, num_id)
        self.mu_2 = mu_2

    def IDcheck(self, passenger):
        """Takes a passenger and checks their ID and boarding pass"""
        yield self.env.timeout(random.expovariate(1.0/self.mu_2))

class PersonalCheck(object):
    """Final passenger processing station. When a passenger arrives,
    they join the shortest queue. When it is their turn to be 
    processed, the time it takes is Uniform(min_pc, max_pc)"""
    def __init__(self, env, NUM_PC, min_pc, max_pc):
        self.env = env
        self.machine = []
        for i in range(NUM_PC):
            self.machine.append(simpy.Resource(env, 1))
        self.min_pc = min_pc
        self.max_pc = max_pc

    def pcheck(self, passenger):
        """The PersonalCheck processes. It takes a passenger and processes them."""
        yield self.env.timeout(random.uniform(self.min_pc,self.max_pc))

def passenger(env, name, cw, ew):
    """The passenger process arrives and requests 
    an IDboarding machine. They then start the checking process,
    waits for it to finish and leaves. They then enter the 
    PersonalCheck station to receive final processing."""
    
    arrive_id=env.now
    with cw.machine.request() as request:
        yield request
        wait_id=env.now-arrive_id
        #print('%s waited %.2f minutes to begin using the IDcheck'% (name, wait_id))
        yield env.process(cw.IDcheck(name))
        
    arrive_pc=env.now
    #Find the shortest wax queue
    minq=0
    for i in range(len(ew.machine)):
        if (ew.machine[i].count < ew.machine[minq].count):
            minq=i
    with ew.machine[minq].request() as request:
        yield request
        wait_pc=env.now-arrive_pc
        #print('%s waited %.2f minutes to begin using the Personal Check'% (name, wait_pc))
        yield env.process(ew.pcheck(name))
        total_wait=wait_id+wait_pc
        TOTAL_WAIT.append(total_wait)

def setup(env, num_id, mu_2, mu_1, num_pc, min_pc, max_pc):
    """Create an idcheck, with passengers approx. 
    every ``mu_1`` minutes."""
    # Create the idcheck and carwax stations
    idcheck = IDboarding(env, num_id, mu_2)
    personalcheck = PersonalCheck(env, num_pc, min_pc, max_pc)
    # Create more passengers while the simulation is running
    i=0
    while True:
        yield env.timeout(np.random.poisson(mu_1))
        i += 1
        env.process(passenger(env, 'Passenger %d' % i, idcheck, personalcheck))


        
#Calculations and Analysis


In [7]:
#Set up a loop to calculate the average over REPLICATIONS:
TOTAL_WAIT=[]
for num_id in range(1,10):
    for num_pc in range(1,10):
        reps=[]
        for i in range(REPLICATIONS):
            random.seed(RANDOM_SEED)  # This helps reproducing the results
            env = simpy.Environment()
            env.process(setup(env, num_id, MU_2, MU_1, num_pc, MIN_PC, MAX_PC))
            env.run(until=SIM_TIME)
            TOTAL_WAIT
            reps.append(np.mean(TOTAL_WAIT))
        print('Average wait time over ', REPLICATIONS, ' replications with num_id=', 
              num_id, " and num_pc=", num_pc, ":",  np.mean(reps))


Average wait time over  10  replications with num_id= 1  and num_pc= 1 : 34.3010667369
Average wait time over  10  replications with num_id= 1  and num_pc= 2 : 34.0633257641
Average wait time over  10  replications with num_id= 1  and num_pc= 3 : 34.0044374835
Average wait time over  10  replications with num_id= 1  and num_pc= 4 : 34.4733403157
Average wait time over  10  replications with num_id= 1  and num_pc= 5 : 34.9509757571
Average wait time over  10  replications with num_id= 1  and num_pc= 6 : 35.2423608078
Average wait time over  10  replications with num_id= 1  and num_pc= 7 : 35.5699376097
Average wait time over  10  replications with num_id= 1  and num_pc= 8 : 35.6766993712
Average wait time over  10  replications with num_id= 1  and num_pc= 9 : 35.7173722404
Average wait time over  10  replications with num_id= 2  and num_pc= 1 : 35.7842448908
Average wait time over  10  replications with num_id= 2  and num_pc= 2 : 34.9993070731
Average wait time over  10  replications wi

In [15]:
#Do the same as above but with the passenger arrival rate of 50 instead of 5
TOTAL_WAIT=[]
for num_id in range(20,40):
    for num_pc in range(20,40):
        reps=[]
        for i in range(REPLICATIONS):
            random.seed(RANDOM_SEED)  # This helps reproducing the results
            env = simpy.Environment()
            env.process(setup(env, num_id, MU_2, 0.02, num_pc, MIN_PC, MAX_PC))
            env.run(until=SIM_TIME)
            reps.append(np.mean(TOTAL_WAIT))
        print('Average wait time over ', REPLICATIONS, ' replications with num_id=', 
              num_id, " and num_pc=", num_pc, ":",  np.mean(reps))

Average wait time over  5  replications with num_id= 20  and num_pc= 20 : 27.0983600861
Average wait time over  5  replications with num_id= 20  and num_pc= 21 : 24.6715140189
Average wait time over  5  replications with num_id= 20  and num_pc= 22 : 24.2441508481
Average wait time over  5  replications with num_id= 20  and num_pc= 23 : 23.4947245061
Average wait time over  5  replications with num_id= 20  and num_pc= 24 : 23.4004827367
Average wait time over  5  replications with num_id= 20  and num_pc= 25 : 23.6365827429
Average wait time over  5  replications with num_id= 20  and num_pc= 26 : 23.680136894
Average wait time over  5  replications with num_id= 20  and num_pc= 27 : 23.5650581524
Average wait time over  5  replications with num_id= 20  and num_pc= 28 : 23.1972113687
Average wait time over  5  replications with num_id= 20  and num_pc= 29 : 23.2234931525
Average wait time over  5  replications with num_id= 20  and num_pc= 30 : 23.035271559
Average wait time over  5  replica

Average wait time over  5  replications with num_id= 24  and num_pc= 34 : 21.3339562624
Average wait time over  5  replications with num_id= 24  and num_pc= 35 : 21.3084722132
Average wait time over  5  replications with num_id= 24  and num_pc= 36 : 21.2711444719
Average wait time over  5  replications with num_id= 24  and num_pc= 37 : 21.2246929902
Average wait time over  5  replications with num_id= 24  and num_pc= 38 : 21.1785725558
Average wait time over  5  replications with num_id= 24  and num_pc= 39 : 21.1729538231
Average wait time over  5  replications with num_id= 25  and num_pc= 20 : 21.155393537
Average wait time over  5  replications with num_id= 25  and num_pc= 21 : 21.1291893039
Average wait time over  5  replications with num_id= 25  and num_pc= 22 : 21.1019961017
Average wait time over  5  replications with num_id= 25  and num_pc= 23 : 21.1017888354
Average wait time over  5  replications with num_id= 25  and num_pc= 24 : 21.084172553
Average wait time over  5  replica

Average wait time over  5  replications with num_id= 29  and num_pc= 28 : 18.4419737046
Average wait time over  5  replications with num_id= 29  and num_pc= 29 : 18.4173147091
Average wait time over  5  replications with num_id= 29  and num_pc= 30 : 18.3892125268
Average wait time over  5  replications with num_id= 29  and num_pc= 31 : 18.3755771884
Average wait time over  5  replications with num_id= 29  and num_pc= 32 : 18.3329177962
Average wait time over  5  replications with num_id= 29  and num_pc= 33 : 18.3010654924
Average wait time over  5  replications with num_id= 29  and num_pc= 34 : 18.2807021386
Average wait time over  5  replications with num_id= 29  and num_pc= 35 : 18.2528807027
Average wait time over  5  replications with num_id= 29  and num_pc= 36 : 18.2183883772
Average wait time over  5  replications with num_id= 29  and num_pc= 37 : 18.1936795379
Average wait time over  5  replications with num_id= 29  and num_pc= 38 : 18.1488679785
Average wait time over  5  repli

Average wait time over  5  replications with num_id= 34  and num_pc= 22 : 15.7461136432
Average wait time over  5  replications with num_id= 34  and num_pc= 23 : 15.7225073524
Average wait time over  5  replications with num_id= 34  and num_pc= 24 : 15.7018309084
Average wait time over  5  replications with num_id= 34  and num_pc= 25 : 15.6786593369
Average wait time over  5  replications with num_id= 34  and num_pc= 26 : 15.6617786721
Average wait time over  5  replications with num_id= 34  and num_pc= 27 : 15.635198351
Average wait time over  5  replications with num_id= 34  and num_pc= 28 : 15.6069437157
Average wait time over  5  replications with num_id= 34  and num_pc= 29 : 15.5865343189
Average wait time over  5  replications with num_id= 34  and num_pc= 30 : 15.5614498313
Average wait time over  5  replications with num_id= 34  and num_pc= 31 : 15.5288203733
Average wait time over  5  replications with num_id= 34  and num_pc= 32 : 15.4945897037
Average wait time over  5  replic